If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/token-classification).

# Fine-tuning a model on a token classification task

In this notebook, we will see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model to a token classification task, which is the task of predicting a label for each token.

![Widget inference representing the NER task](images/token_classification.png)

The most common token classification tasks are:

- NER (Named-entity recognition) Classify the entities in the text (person, organization, location...).
- POS (Part-of-speech tagging) Grammatically classify the tokens (noun, verb, adjective...)
- Chunk (Chunking) Grammatically classify the tokens and group them into "chunks" that go together

We will see how to easily load a dataset for these kinds of tasks and use the `Trainer` API to fine-tune a model on it.

This notebook is built to run on any token classification task, with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a version with a token classification head and a fast tokenizer (check on [this table](https://huggingface.co/transformers/index.html#bigtable) if this is the case). It might just need some small adjustments if you decide to use a different dataset than the one used here. Depending on you model and the GPU you are using, you might need to adjust the batch size to avoid out-of-memory errors. Set those three parameters, then the rest of the notebook should run smoothly:

In [36]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "camembert-base"
batch_size = 32

## Loading the dataset

We will use the [🤗 Datasets](https://github.com/huggingface/datasets) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and `load_metric`.  

In [37]:
from datasets import load_dataset, load_metric

For our example here, we'll use the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf). The notebook should work with any token classification dataset provided by the 🤗 Datasets library. If you're using your own dataset defined from a JSON or csv file (see the [Datasets documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) on how to load them), it might need some adjustments in the names of the columns used.

In [3]:
datasets = load_dataset('presto.py', data_files={'train': 'presto/train.conll', 'validation': 'presto/dev.conll', 'test': 'presto/test.conll'})

Using custom data configuration default-9718515ffafdfc6c
Reusing dataset presto (/gpfswork/rech/wya/upf12cf/.cache/huggingface/datasets/presto/default-9718515ffafdfc6c/0.0.0/d5455c089e3f2a14e8357d6b2178f6c1182d99c72e163bd75f403db573fb7a1a)


The `datasets` object itself is [`DatasetDict`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set.

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 112887
    })
    validation: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 14109
    })
    test: Dataset({
        features: ['id', 'ner_tags', 'tokens'],
        num_rows: 14117
    })
})

We can see the training, validation and test sets all have a column for the tokens (the input texts split into words) and one column of labels for each kind of task we introduced before.

To access an actual element, you need to select a split first, then give an index:

In [5]:
datasets["train"][0]

{'id': '0',
 'ner_tags': [4,
  14,
  14,
  14,
  4,
  4,
  14,
  14,
  14,
  14,
  14,
  14,
  4,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14,
  14],
 'tokens': ['GRIMALTE',
  'AMANT',
  'DE',
  'LA',
  'dame',
  'Gradisse',
  'narre',
  'sommairement',
  'Les',
  'amoureux',
  'regredz',
  'de',
  'Flamete',
  ',',
  'qui',
  'furent',
  'occasion',
  "qu'",
  'il',
  'cherchast',
  'moyen',
  "d'",
  'y',
  'remedier',
  'Chapitre',
  'premier',
  '.']}

The labels are already coded as integer ids to be easily usable by our model, but the correspondence with the actual categories is stored in the `features` of the dataset:

In [6]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=15, names=['B-event', 'B-func', 'B-loc', 'B-org', 'B-pers', 'B-prod', 'B-time', 'I-event', 'I-func', 'I-loc', 'I-org', 'I-pers', 'I-prod', 'I-time', 'O'], names_file=None, id=None), length=-1, id=None)

So for the NER tags, 0 corresponds to 'O', 1 to 'B-PER' etc... On top of the 'O' (which means no special entity), there are four labels for NER here, each prefixed with 'B-' (for beginning) or 'I-' (for intermediate), that indicate if the token is the first one for the current group with the label or not:
- 'PER' for person
- 'ORG' for organization
- 'LOC' for location
- 'MISC' for miscellaneous

Since the labels are lists of `ClassLabel`, the actual names of the labels are nested in the `feature` attribute of the object above:

In [7]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['B-event',
 'B-func',
 'B-loc',
 'B-org',
 'B-pers',
 'B-prod',
 'B-time',
 'I-event',
 'I-func',
 'I-loc',
 'I-org',
 'I-pers',
 'I-prod',
 'I-time',
 'O']

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset (automatically decoding the labels in passing).

## Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a 🤗 Transformers `Tokenizer` which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the `AutoTokenizer.from_pretrained` method, which will ensure:

- we get a tokenizer that corresponds to the model architecture we want to use,
- we download the vocabulary used when pretraining this specific checkpoint.

That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

In [8]:
from transformers import CamembertTokenizerFast
    
tokenizer = CamembertTokenizerFast.from_pretrained(model_checkpoint, local_files_only=True)

The following assertion ensures that our tokenizer is a fast tokenizers (backed by Rust) from the 🤗 Tokenizers library. Those fast tokenizers are available for almost all models, and we will need some of the special features they have for our preprocessing.

In [9]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You can check which type of models have a fast tokenizer available and which don't on the [big table of models](https://huggingface.co/transformers/index.html#bigtable).

You can directly call this tokenizer on one sentence:

In [10]:
tokenizer("Bonjour, voici un phrase !")

{'input_ids': [5, 1285, 7, 1510, 23, 3572, 83, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Depending on the model you selected, you will see different keys in the dictionary returned by the cell above. They don't matter much for what we're doing here (just know they are required by the model we will instantiate later), you can learn more about them in [this tutorial](https://huggingface.co/transformers/preprocessing.html) if you're interested.

If, as is the case here, your inputs have already been split into words, you should pass the list of words to your tokenzier with the argument `is_split_into_words=True`:

In [11]:
tokenizer(["Bonjour", ",", "voici", "une", "prhase", "découpée", "en", "mots", "!"], is_split_into_words=True)

{'input_ids': [5, 1285, 21, 7, 1510, 28, 2223, 133, 6996, 13829, 35, 22, 883, 83, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Note that transformers are often pretrained with subword tokenizers, meaning that even if your inputs have been split into words already, each of those words could be split again by the tokenizer. Let's look at an example of that:

In [12]:
example = datasets["train"][4]
print(example["tokens"])

['De', 'laquelle', 'choſe', 'il', "n'", 'eust', 'pouvoir', '/', 'ou', 'vouloir', 'de', 'maintenir', '.']


In [13]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '▁De', '▁laquelle', '▁chose', '▁il', '▁n', "'", '▁eu', 'st', '▁pouvoir', '▁/', '▁ou', '▁vouloir', '▁de', '▁maintenir', '▁', '.', '</s>']


Here the words "Zwingmann" and "sheepmeat" have been split in three subtokens.

This means that we need to do some processing on our labels as the input ids returned by the tokenizer are longer than the lists of labels our dataset contain, first because some special tokens might be added (we can a `[CLS]` and a `[SEP]` above) and then because of those possible splits of words in multiple tokens:

In [14]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(13, 18)

Thankfully, the tokenizer returns outputs that have a `word_ids` method which can help us.

In [15]:
print(tokenized_input.word_ids())

[None, 0, 1, 2, 3, 4, 4, 5, 5, 6, 7, 8, 9, 10, 11, 12, 12, None]


As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to `None` and all other tokens to their respective word. This way, we can align the labels with the processed input ids.

In [16]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

18 18


Here we set the labels of all special tokens to -100 (the index that is ignored by PyTorch) and the labels of all other tokens to the label of the word they come from. Another strategy is to set the label only on the first token obtained from a given word, and give a label of -100 to the other subtokens from the same word. We propose the two strategies here, just change the value of the following flag:

In [17]:
label_all_tokens = True

We're now ready to write the function that will preprocess our samples. We feed them to the `tokenizer` with the argument `truncation=True` (to truncate texts that are bigger than the maximum size allowed by the model) and `is_split_into_words=True` (as seen above). Then we align the labels with the token ids using the strategy we picked:

In [18]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

This function works with one or several examples. In the case of several examples, the tokenizer will return a list of lists for each key:

In [19]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[5, 7348, 4167, 3290, 3219, 9207, 5669, 988, 1925, 5942, 29444, 2465, 49, 12965, 15725, 131, 74, 2800, 16333, 9056, 138, 8, 17589, 10276, 21, 7, 31, 3180, 1932, 46, 11, 51, 661, 751, 4380, 694, 18, 11, 102, 9264, 35, 13558, 14015, 246, 21, 9, 6], [5, 8543, 521, 362, 5957, 1151, 5633, 37, 100, 3143, 8, 27405, 10, 24, 966, 1738, 55, 58, 373, 22, 18408, 341, 487, 21, 7, 7066, 17, 11, 14380, 30, 12576, 13, 17589, 4580, 35, 5361, 35, 37, 261, 24304, 291, 21, 7, 537, 126, 6712, 663, 27, 44, 1137, 290, 31, 44, 3369, 11334, 1171, 37, 3874, 30751, 49, 11, 2890, 27450, 77, 17393, 573, 35, 37, 599, 17741, 21, 7, 17, 11, 33, 105, 72, 1913, 1723, 3369, 8, 14780, 108, 15725, 131, 21, 7, 264, 27, 163, 30, 268, 27, 119, 487, 5942, 8704, 110, 17, 11, 28, 8, 989, 21, 7, 31, 674, 33, 491, 537, 8489, 6141, 19, 36, 219, 4597, 1017, 291, 1233, 312, 21, 7, 37, 163, 109, 30, 172, 1113, 3568, 5282, 487, 22, 1103, 42, 2145, 58, 19158, 292, 105, 48, 343, 11347, 10875, 13, 40, 4125, 8, 58, 271, 3, 

To apply this function on all the sentences (or pairs of sentences) in our dataset, we just use the `map` method of our `dataset` object we created earlier. This will apply the function on all the elements of all the splits in `dataset`, so our training, validation and testing data will be preprocessed in one single command.

In [20]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /gpfswork/rech/wya/upf12cf/.cache/huggingface/datasets/presto/default-9718515ffafdfc6c/0.0.0/d5455c089e3f2a14e8357d6b2178f6c1182d99c72e163bd75f403db573fb7a1a/cache-b48e32a1b2304f06.arrow
Loading cached processed dataset at /gpfswork/rech/wya/upf12cf/.cache/huggingface/datasets/presto/default-9718515ffafdfc6c/0.0.0/d5455c089e3f2a14e8357d6b2178f6c1182d99c72e163bd75f403db573fb7a1a/cache-559a1d261bbaa18c.arrow
Loading cached processed dataset at /gpfswork/rech/wya/upf12cf/.cache/huggingface/datasets/presto/default-9718515ffafdfc6c/0.0.0/d5455c089e3f2a14e8357d6b2178f6c1182d99c72e163bd75f403db573fb7a1a/cache-63b758c779c14a53.arrow


Even better, the results are automatically cached by the 🤗 Datasets library to avoid spending time on this step the next time you run your notebook. The 🤗 Datasets library is normally smart enough to detect when the function you pass to map has changed (and thus requires to not use the cache data). For instance, it will properly detect if you change the task in the first cell and rerun the notebook. 🤗 Datasets warns you when it uses cached files, you can pass `load_from_cache_file=False` in the call to `map` to not use the cached files and force the preprocessing to be applied again.

Note that we passed `batched=True` to encode the texts by batches together. This is to leverage the full benefit of the fast tokenizer we loaded earlier, which will use multi-threading to treat the texts in a batch concurrently.

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since all our tasks are about token classification, we use the `AutoModelForTokenClassification` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us. The only thing we have to specify is the number of labels for our problem (which we can get from the features, as seen before):

In [21]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), local_files_only=True)

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertForTokenClassification: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream tas

The warning is telling us we are throwing away some weights (the `vocab_transform` and `vocab_layer_norm` layers) and randomly initializing some other (the `pre_classifier` and `classifier` layers). This is absolutely normal in this case, because we are removing the head used to pretrain the model on a masked language modeling objective and replacing it with a new head for which we don't have pretrained weights, so the library warns us we should fine-tune this model before using it for inference, which is exactly what we are going to do.

To instantiate a `Trainer`, we will need to define three more things. The most important is the [`TrainingArguments`](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [27]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
)

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the `batch_size` defined at the top of the notebook and customize the number of epochs for training, as well as the weight decay.

Then we will need a data collator that will batch our processed examples together while applying padding to make them all the same size (each pad will be padded to the length of its longest example). There is a data collator for this task in the Transformers library, that not only pads the inputs, but also the labels:

In [28]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

The last thing to define for our `Trainer` is how to compute the metrics from the predictions. Here we will load the [`seqeval`](https://github.com/chakki-works/seqeval) metric (which is commonly used to evaluate results on the CONLL dataset) via the Datasets library.

In [29]:
metric = load_metric("seqeval")

Using the latest cached version of the module from /gpfswork/rech/wya/upf12cf/.cache/huggingface/modules/datasets_modules/metrics/seqeval/1fde2544ac1f3f7e54c639c73221d3a5e5377d2213b9b0fdb579b96980b84b2e (last modified on Wed Jun 23 23:26:12 2021) since it couldn't be found locally at seqeval/seqeval.py or remotely (ConnectionError).


This metric takes list of labels for the predictions and references:

In [30]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

So we will need to do a bit of post-processing on our predictions:
- select the predicted index (with the maximum logit) for each token
- convert it to its string label
- ignore everywhere we set a label of -100

The following function does all this post-processing on the result of `Trainer.evaluate` (which is a namedtuple containing predictions and labels) before applying the metric:

In [31]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

Note that we drop the precision/recall/f1 computed for each category and only focus on the overall precision/recall/f1/accuracy.

Then we just need to pass all of this along with our datasets to the `Trainer`:

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

We can now finetune our model by just calling the `train` method:

In [33]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running training *****
  Num examples = 112887
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 8820
/gpfswork/rech/wya/upf12cf/miniconda3/envs/transformers/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.269505,0.627647,0.751542,0.684030,0.980713
2,0.637900,0.104803,0.776978,0.829776,0.802509,0.987435
3,0.119600,0.078112,0.808159,0.822481,0.815257,0.988474
4,0.059200,0.065302,0.820085,0.849554,0.834560,0.989256
5,0.039700,0.058515,0.826624,0.855478,0.840804,0.989561
6,0.030000,0.057940,0.822483,0.862185,0.841866,0.989527
7,0.023500,0.060357,0.830175,0.842896,0.836487,0.989349
8,0.019700,0.060739,0.840533,0.834525,0.837518,0.989555
9,0.019700,0.052468,0.829418,0.866004,0.847316,0.989898
10,0.016300,0.055727,0.838183,0.851807,0.844940,0.990043


The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running Evaluation *****
  Num examples = 14109
  Batch size = 256
/gpfswork/rech/wya/upf12cf/miniconda3/envs/transformers/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
/gpfswork/rech/wya/upf12cf/miniconda3/envs/transformers/lib/python3.8/site-packages/to

TrainOutput(global_step=8820, training_loss=0.05892363735337376, metrics={'train_runtime': 6262.7772, 'train_samples_per_second': 360.501, 'train_steps_per_second': 1.408, 'total_flos': 4.5778066342544954e+17, 'train_loss': 0.05892363735337376, 'epoch': 20.0})

The `evaluate` method allows you to evaluate again on the evaluation dataset or on another dataset:

In [34]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running Evaluation *****
  Num examples = 14109
  Batch size = 256


{'eval_loss': 0.05803868547081947,
 'eval_precision': 0.8379556449681468,
 'eval_recall': 0.8564574561832958,
 'eval_f1': 0.8471055371280536,
 'eval_accuracy': 0.9901027707120542,
 'eval_runtime': 24.1439,
 'eval_samples_per_second': 584.371,
 'eval_steps_per_second': 2.319,
 'epoch': 20.0}

To get the precision/recall/f1 computed for each category now that we have finished training, we can apply the same function as before on the result of the `predict` method:

In [35]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, id, ner_tags.
***** Running Prediction *****
  Num examples = 14109
  Batch size = 256


{'event': {'precision': 1.0,
  'recall': 0.9565217391304348,
  'f1': 0.9777777777777777,
  'number': 23},
 'func': {'precision': 0.7508771929824561,
  'recall': 0.9029535864978903,
  'f1': 0.8199233716475096,
  'number': 237},
 'loc': {'precision': 0.7897954880877082,
  'recall': 0.8033454857387947,
  'f1': 0.7965128641292791,
  'number': 4663},
 'org': {'precision': 0.7214765100671141,
  'recall': 0.8958333333333334,
  'f1': 0.7992565055762082,
  'number': 240},
 'pers': {'precision': 0.8557467309015829,
  'recall': 0.8683567288218451,
  'f1': 0.8620056154459427,
  'number': 14319},
 'prod': {'precision': 0.7671957671957672,
  'recall': 0.7712765957446809,
  'f1': 0.7692307692307693,
  'number': 188},
 'time': {'precision': 0.8864197530864197,
  'recall': 0.9497354497354498,
  'f1': 0.9169859514687101,
  'number': 756},
 'overall_precision': 0.8379556449681468,
 'overall_recall': 0.8564574561832958,
 'overall_f1': 0.8471055371280536,
 'overall_accuracy': 0.9901027707120542}

Don't forget to [upload your model](https://huggingface.co/transformers/model_sharing.html) on the [🤗 Model Hub](https://huggingface.co/models). You can then use it only to generate results like the one shown in the first picture of this notebook!